In [1]:
from databroker import db, get_table

from metadatastore.commands import (insert_run_start, find_run_starts,
                                    insert_descriptor, find_descriptors,
                                    insert_event, find_events,
                                    insert_run_stop, find_run_stops,
                                    descriptors_by_start, get_events_generator)

from ixstools.io import Specfile
import uuid
import os
from metadatastore.utils.testing import mds_setup, mds_teardown

In [2]:
mds_setup()

In [3]:
specpath = '/home/edill/dev/python/ixstools/ixstools/'
sf = Specfile(specpath + os.sep + '20160219.spec')

In [4]:
def run_start(specscan, beamline_id, **md):
    run_start_dict = {
        'time': specscan.time_from_date.timestamp(),
        'scan_id': specscan.scan_id,
        'beamline_id': beamline_id,
        'uid': str(uuid.uuid4()),
        'specpath': specscan.specfile.filename,
        'owner': specscan.specfile.parsed_header['user'],
        'plan_args': specscan.scan_args,
        'scan_type': specscan.scan_command,
    }
    run_start_dict.update(**md)
    return insert_run_start(**run_start_dict)

def baseline(specscan, start_uid):
    timestamp = specscan.time_from_date.timestamp()
    data_keys = {}
    data = {}
    timestamps = {}
    for obj_name, human_name, value in zip(specscan.specfile.parsed_header['motor_spec_names'], 
                                           specscan.specfile.parsed_header['motor_human_names'], 
                                           specscan.motor_values):
        data_keys[obj_name] = {'dtype': 'number', 'shape': [], 'source': human_name}
        data[obj_name] = value
        timestamps[obj_name] = timestamp
    data_keys.update({k: {'dtype': 'number', 'shape': [], 'source': k} for k in 'hkl'})
    data.update({k: v for k, v in zip('hkl', specscan.hkl)})
    timestamps.update({k: timestamp for k in 'hkl'})
    descriptor_uid = insert_descriptor(run_start=start_uid, data_keys=data_keys, 
                                       time=timestamp, uid=str(uuid.uuid4()), name='baseline')
    yield 'descriptor', descriptor_uid
    yield 'event', insert_event(descriptor=descriptor_uid, seq_num=0, time=timestamp,
                                data=data, timestamps=timestamps, uid=str(uuid.uuid4()))

def events(specscan, start_uid):
    timestamp = specscan.time_from_date.timestamp()
    data_keys = {}
    data = {}
    timestamps = {}
    
    data_keys = {col: {'dtype': 'number', 'shape': [], 'source': col} for col in specscan.col_names}
    descriptor_uid = insert_descriptor(run_start=start_uid, data_keys=data_keys,
                                       time=timestamp, uid=str(uuid.uuid4()),
                                       name='primary')
    yield 'descriptor', descriptor_uid
    timestamps = {col: timestamp for col in specscan.col_names}
    for seq_num, (x, row_series) in enumerate(specscan.scan_data.iterrows()):
        data = {col: data for col, data in zip(row_series.index, row_series[:])}
        yield 'event', insert_event(data=data, descriptor=descriptor_uid, seq_num=seq_num,
                                    time=timestamp + data['Epoch'],
                                    timestamps=timestamps, uid=str(uuid.uuid4()))

def stop(specscan, start_uid, **md):
    timestamp = specscan.time_from_date.timestamp()
    yield 'stop', insert_run_stop(run_start=start_uid, time=timestamp, uid=str(uuid.uuid4()), **md)

In [5]:
def to_document_stream(specscan, beamline_id):
    start_uid = run_start(specscan, beamline_id)
    yield 'start', start_uid
    yield from baseline(specscan, start_uid)
    yield from events(specscan, start_uid)
    # do some sanity checks
    descriptors = descriptors_by_start(start_uid)
    assert len(descriptors) == 2
    baseline_descriptor, primary_descriptor = descriptors
    if baseline_descriptor.name != 'baseline':
        baseline_descriptor, primary_descriptor = primary_descriptor, baseline_descriptor
    baseline_descriptor = descriptors[0] if descriptors[0].name == 'baseline' else descriptors[1]
    baseline_events = list(get_events_generator(baseline_descriptor))
    assert len(baseline_events) == 1
    primary_descriptor = descriptors[0] if descriptors[0].name == 'primary' else descriptors[1]
    primary_events = list(get_events_generator(primary_descriptor))
    if len(primary_events) <= specscan.num_points:
        print('scan %s only has %s/%s points. Assuming scan was aborted. start_uid=%s' 
              % (specscan.scan_id, len(primary_events), specscan.num_points, start_uid))
        reason = 'abort'
    else:
        reason = 'success'
    yield from stop(specscan, start_uid, reason=reason)
    

In [6]:
for specscan in sf:
    stream = list(to_document_stream(specscan, 'ixs'))

scan 1 only has 12/34 points. Assuming scan was aborted. start_uid=d87b6ccf-d45e-485f-b933-3caedc95438e
scan 2 only has 31/34 points. Assuming scan was aborted. start_uid=863e687f-4b56-4dcf-a6d2-6af695493e15
scan 14 only has 18/34 points. Assuming scan was aborted. start_uid=c7a01787-97ee-4cba-a1dc-0f06908e7e06
scan 15 only has 15/34 points. Assuming scan was aborted. start_uid=6e1d36b2-dfa9-4c1a-a575-4d54bbb4f4fd
scan 33 only has 23/31 points. Assuming scan was aborted. start_uid=e13bd8b6-c394-4c0c-a70f-825eae1f34f3
scan 34 only has 28/31 points. Assuming scan was aborted. start_uid=9eb2bcf9-913d-4131-bf4d-de0b371349a3


In [7]:
headers = db(specpath='/home/edill/dev/python/ixstools/ixstools/20160219.spec')

In [8]:
assert len(headers) == len(sf)

In [9]:
len(sf)

34

In [10]:
len(headers)

34

In [11]:
# show the contents of the document stream
stream

[('start', '9eb2bcf9-913d-4131-bf4d-de0b371349a3'),
 ('descriptor', '71353344-a66b-404a-bf7d-c794f2a7a06c'),
 ('event', '450f68e0-b996-4d10-b9a4-59e234940b00'),
 ('descriptor', '821db274-6dd4-4a1d-9262-2d06b269a611'),
 ('event', '46c89451-7da7-422f-b06f-c0a486eb4fb5'),
 ('event', 'd61d0c27-0dba-431b-a8e4-78acb1401274'),
 ('event', 'b9070f46-e188-4c56-9e48-b28c4e538ebb'),
 ('event', '1c910a55-1ab7-4b5b-b587-26c1e1fe244d'),
 ('event', '53773a34-6d8b-45d5-9976-2a64272f8ec5'),
 ('event', 'aa8cf2a0-c903-4c0a-bde0-2851a9bb4c9c'),
 ('event', 'e3853bf2-a13e-4517-8b71-5001663e7482'),
 ('event', '820f76b3-c17d-4aa6-9328-8a0223e3fd48'),
 ('event', '71eae70b-8016-4b44-97cf-b2685558948e'),
 ('event', '9539451f-742b-4728-aa1c-2643df7e57d1'),
 ('event', '6c6b5f7b-7f48-4f3b-9b9c-3ce4f31b23ff'),
 ('event', 'b08d72b1-984b-4ec3-9f2c-23d8279c8ffc'),
 ('event', '1fe5ee64-01c4-4552-b5b4-d10f465b3678'),
 ('event', '9c08fc98-6066-4e59-8a4a-f66cfb34f87c'),
 ('event', 'b7e41000-3ace-4249-866e-04481cfbc90f'),
 (

In [12]:
db[stream[0][1]].start

beamline_id,ixs
group,
owner,asuvorov
plan_args,"['1', '0']"
project,
sample,
scan_id,34
scan_type,timescan
specpath,/home/edill/dev/python/ixstools/ixstools/20160219.spec
time,a month ago (2016-02-22T13:54:26)
uid,9eb2bcf9-913d-4131-bf4d-de0b371349a3


In [13]:
print(db[stream[0][1]].descriptors[0])


EventDescriptor
+-----------+--------+-------+----------+
| data keys | dtype  | shape |  source  |
+-----------+--------+-------+----------+
| Detector  | number |   []  | Detector |
| Epoch     | number |   []  |  Epoch   |
| Monitor   | number |   []  | Monitor  |
| PD11      | number |   []  |   PD11   |
| PD12      | number |   []  |   PD12   |
| PD13      | number |   []  |   PD13   |
| PD14      | number |   []  |   PD14   |
| PD21      | number |   []  |   PD21   |
| PD22      | number |   []  |   PD22   |
| PD23      | number |   []  |   PD23   |
| PD24      | number |   []  |   PD24   |
| PD31      | number |   []  |   PD31   |
| PD32      | number |   []  |   PD32   |
| PD33      | number |   []  |   PD33   |
| PD34      | number |   []  |   PD34   |
| PD41      | number |   []  |   PD41   |
| PD42      | number |   []  |   PD42   |
| PD43      | number |   []  |   PD43   |
| PD44      | number |   []  |   PD44   |
| PD51      | number |   []  |   PD51   |
| PD52      | num

In [14]:
print(db[stream[0][1]].descriptors[1])


EventDescriptor
+-----------+--------+-------+-----------+
| data keys | dtype  | shape |   source  |
+-----------+--------+-------+-----------+
| acchi     | number |   []  |  ANA_CChi |
| acfth     | number |   []  |  ANA_CFTh |
| acth      | number |   []  |  ANA_CThe |
| acyy      | number |   []  |   ANA_CY  |
| adth      | number |   []  |  ANA_DTH  |
| adyy      | number |   []  |  ANA_DYY  |
| anay      | number |   []  |   ANA_AY  |
| anby      | number |   []  |   ANA_BY  |
| andy      | number |   []  |   ANA_DY  |
| anpd      | number |   []  |  ANA_PDCW |
| anuy      | number |   []  |   ANA_UY  |
| awchi     | number |   []  |  ANA_WChi |
| awfth     | number |   []  |  ANA_WFTh |
| bicd      | number |   []  |    BICD   |
| bicu      | number |   []  |    BICU   |
| blE       | number |   []  |   BL_EN   |
| bpm1_p    | number |   []  |   BPM1_P  |
| bpm1dx    | number |   []  |  BPM1_DX  |
| bpm1dy    | number |   []  |  BPM1_DY  |
| bpm1y     | number |   []  |   BPM1

In [15]:
print(db[stream[0][1]].stop)


RunStop
exit_status     : success                                 
reason          : abort                                   
run_start       : 9eb2bcf9-913d-4131-bf4d-de0b371349a3    
time            : 1456167266.0                            
uid             : 2502b44a-f7cc-4863-a679-6f9d85278697    


In [16]:
mds_teardown()